In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [8]:
path = "/content/drive/My Drive/Colab Notebooks/data/training.1600000.processed.noemoticon.csv"

In [10]:
entire_sentiment140_df = pd.read_csv(path, header=None, encoding="latin")

In [27]:
some_negative_tweets_df = entire_sentiment140_df.head(500)
some_negative_tweets_df

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,Awww thats bummer You shoulda got David C...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset that cant update his Facebook texting...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,dived many times for the ball Managed save ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feels itchy and like its fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,its not behaving all mad why here becaus...
...,...,...,...,...,...,...
495,0,1467934606,Mon Apr 06 22:53:06 PDT 2009,NO_QUERY,fictillius,geez fun are you
496,0,1467935121,Mon Apr 06 22:53:14 PDT 2009,NO_QUERY,SicklyInfected,wanna punk rock band again
497,0,1467935189,Mon Apr 06 22:53:15 PDT 2009,NO_QUERY,AshleighF,Thats lot angst for Tuesday afternoon
498,0,1467935271,Mon Apr 06 22:53:19 PDT 2009,NO_QUERY,andreakuefler,very glad Britney isnt crazy anymore That was...


In [28]:
some_positive_tweets_df = entire_sentiment140_df.tail(500)
some_positive_tweets_df

,0,1,2,3,4,5
1599500,4,2193453768,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,lyndiclark,Sitting meeting with mom grandmother regua...
1599501,4,2193453784,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,_cammi_,getting used twitter
1599502,4,2193453791,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,nikibrown,hackorz thats teh illegalz noes howd you
1599503,4,2193453828,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,cocoletteng,Wilde brilliant century writerpoet Harpers ...
1599504,4,2193453834,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,Codieeee,Love She AMAZINMy favourite singerI Only ju...
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke Having school the best feeling ever
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDBcom Very cool hear old Walt interviews
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover Ask for ...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy Birthday boo alll time Tupac Amaru S...


In [20]:
import re

def remove_urls(tweet):
  return re.sub(r'www\.\S+|https?://\S+', '', tweet)

def remove_html_character_entities(tweet):
  return re.sub(r'&[a-zA-Z]+;', '', tweet)

def remove_at_mentions(tweet):
  return re.sub(r'@\S+', '', tweet)

def remove_non_alpha_or_space_characters(tweet):        
    return re.sub(r'[^a-zA-Z\s]', '', tweet)

def remove_short_words(tweet):
  return re.sub(r'\b\w{1,2}\b', '', tweet)

def preprocess_tweet(tweet):
  tweet = remove_urls(tweet)
  tweet = remove_html_character_entities(tweet)
  tweet = remove_at_mentions(tweet)
  tweet = remove_non_alpha_or_space_characters(tweet)
  return remove_short_words(tweet)

In [30]:
some_negative_tweets_df[5] = some_negative_tweets_df[5].map(preprocess_tweet)
some_negative_tweets_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,Awww thats bummer You shoulda got David C...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset that cant update his Facebook texting...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,dived many times for the ball Managed save ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feels itchy and like its fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,its not behaving all mad why here becaus...
...,...,...,...,...,...,...
495,0,1467934606,Mon Apr 06 22:53:06 PDT 2009,NO_QUERY,fictillius,geez fun are you
496,0,1467935121,Mon Apr 06 22:53:14 PDT 2009,NO_QUERY,SicklyInfected,wanna punk rock band again
497,0,1467935189,Mon Apr 06 22:53:15 PDT 2009,NO_QUERY,AshleighF,Thats lot angst for Tuesday afternoon
498,0,1467935271,Mon Apr 06 22:53:19 PDT 2009,NO_QUERY,andreakuefler,very glad Britney isnt crazy anymore That was...


In [31]:
some_positive_tweets_df[5] = some_positive_tweets_df[5].map(preprocess_tweet)
some_positive_tweets_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,0,1,2,3,4,5
1599500,4,2193453768,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,lyndiclark,Sitting meeting with mom grandmother regua...
1599501,4,2193453784,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,_cammi_,getting used twitter
1599502,4,2193453791,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,nikibrown,hackorz thats teh illegalz noes howd you
1599503,4,2193453828,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,cocoletteng,Wilde brilliant century writerpoet Harpers ...
1599504,4,2193453834,Tue Jun 16 08:28:49 PDT 2009,NO_QUERY,Codieeee,Love She AMAZINMy favourite singerI Only ju...
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke Having school the best feeling ever
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDBcom Very cool hear old Walt interviews
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover Ask for ...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy Birthday boo alll time Tupac Amaru S...


In [32]:
all_training_tweets_df = pd.concat([some_negative_tweets_df, some_positive_tweets_df], ignore_index=True)
all_training_tweets_df

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,Awww thats bummer You shoulda got David C...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset that cant update his Facebook texting...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,dived many times for the ball Managed save ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feels itchy and like its fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,its not behaving all mad why here becaus...
...,...,...,...,...,...,...
995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke Having school the best feeling ever
996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDBcom Very cool hear old Walt interviews
997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover Ask for ...
998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy Birthday boo alll time Tupac Amaru S...


In [33]:
pip install --upgrade scikit-learn

     |████████████████████████████████| 6.8MB 2.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [34]:
from sklearn.utils import shuffle

all_training_tweets_df = shuffle(all_training_tweets_df)
all_training_tweets_df.reset_index(inplace=True, drop=True)
all_training_tweets_df

,0,1,2,3,4,5
0,4,2193476528,Tue Jun 16 08:30:37 PDT 2009,NO_QUERY,heatherzajac,hiking the indian council cave trail today
1,0,1467891880,Mon Apr 06 22:41:06 PDT 2009,NO_QUERY,Ps8612,Argh was suuuper sleepy hour ago now wide a...
2,0,1467895424,Mon Apr 06 22:42:07 PDT 2009,NO_QUERY,MissPassion,the suns already gone
3,0,1467879314,Mon Apr 06 22:37:46 PDT 2009,NO_QUERY,sarochka,Aaaaand the nausea back
4,0,1467893730,Mon Apr 06 22:41:36 PDT 2009,NO_QUERY,lisha_e,cuuute hey miss you its not the same not ...
...,...,...,...,...,...,...
995,0,1467855673,Mon Apr 06 22:31:29 PDT 2009,NO_QUERY,IluvJoelMadden,Going school soon cant find anything wear go...
996,4,2193473935,Tue Jun 16 08:30:25 PDT 2009,NO_QUERY,wrappedin,drive through rainbows pretty pretty
997,4,2193474811,Tue Jun 16 08:30:29 PDT 2009,NO_QUERY,sweetbratz16,hours and then
998,4,2193528067,Tue Jun 16 08:34:48 PDT 2009,NO_QUERY,Natalie_1990,cant wait for born run


In [40]:
# Didn't work in Colab initially; but later it did; I think it was to do w/ old version of sklearn
from nltk.corpus import stopwords
#stopwords.words('english')

# import nltk
# nltk.download('stopwords')

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_classifier = Pipeline([
  ('vectorizer', HashingVectorizer(stop_words=stopwords.words('english'),
                                   alternate_sign=False)),
  ('tfidf_transformer', TfidfTransformer()),
  ('classifier', MultinomialNB()),
])

In [42]:
from sklearn.model_selection import GridSearchCV

# part before double underscore needs to be same as classifier name
# See https://stackoverflow.com/questions/48271342/invalid-parameter-clf-for-estimator-pipeline-in-sklearn
parameters = {      
  'classifier__alpha': (1e-2, 1e-3), 
}

grid_search_classifier = GridSearchCV(text_classifier, parameters, cv=2, n_jobs=-2)

In [45]:
import time

training_samples = all_training_tweets_df.iloc[:,-1] 
training_labels = all_training_tweets_df.iloc[:,0]
tic = time.perf_counter()
grid_search_classifier = grid_search_classifier.fit(training_samples, training_labels)
toc = time.perf_counter()
print(f"Took {toc - tic:0.4f} seconds to fit classifer on data")

Took 0.9880 seconds to fit classifer on data


In [46]:
print(f"Best score: {grid_search_classifier.best_score_}")

Best score: 0.613


In [48]:
import pickle
with open('test_grid_search_NB_clf_sentiment140.pkl', 'wb') as f:
  pickle.dump(grid_search_classifier, f)